In [27]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [28]:
datadir = '/cellar/users/snwright/Data/RareCommon/GWASAtlas'

In [29]:
overlap = set(info['PMID'].astype(str).unique()).intersection(set(cat_info['PUBMED ID'].astype(str).unique()))
len(overlap)

327

In [30]:
a = cat_info[cat_info['PUBMED ID'].isin([int(x) for x in overlap])]

In [31]:
b = info[info['PMID'].isin([str(x) for x in overlap])]

## GWASAtlas

In [32]:
info = pd.read_csv(os.path.join(datadir, 'gwasATLAS_v20191115.txt.gz' ), sep='\t', low_memory=False, index_col=0)
info.shape
info['ga_idx'] = [x for x in range(len(info))]

In [33]:
info.loc[1141]

PMID                                                         27494321
Year                                                             2016
File                http://www.t2diabetesgenes.org/datasets/Chrono...
Website                          http://www.t2diabetesgenes.org/data/
Consortium                                                  T2D-GENES
Domain                                                    Psychiatric
ChapterLevel                                         Mental Functions
SubchapterLevel                                       Sleep Functions
Trait                                                      Chronotype
uniqTrait                                                  Chronotype
Population                                                 UKB1 (EUR)
Ncase                                                             NaN
Ncontrol                                                          NaN
N                                                              128266
Genome              

In [34]:
print(f'Unique domains: {info.Domain.nunique()}')
print(f'Unique chapters: {info.ChapterLevel.nunique()}')
print(f'Unique subchapters: {info.SubchapterLevel.nunique()}')
print(f'Unique traits: {info.Trait.nunique()}')
print(f'Unique UniqTraits: {info.uniqTrait.nunique()}')
print(f'Unique PMID: {info.PMID.nunique()}')
print(f'Unique Consortium: {info.Consortium.nunique()}')
print(f'Unique Populations: {info.Population.nunique()}')

Unique domains: 28
Unique chapters: 55
Unique subchapters: 209
Unique traits: 4363
Unique UniqTraits: 3307
Unique PMID: 473
Unique Consortium: 40
Unique Populations: 34


In [35]:
ukb_ids = ['UKB2 (EUR)', 'UKB1 (EUR)']

In [36]:
ukb_info = info[info.Population.isin(ukb_ids)]

In [37]:
print(f'Unique domains: {ukb_info.Domain.nunique()}')
print(f'Unique chapters: {ukb_info.ChapterLevel.nunique()}')
print(f'Unique subchapters: {ukb_info.SubchapterLevel.nunique()}')
print(f'Unique traits: {ukb_info.Trait.nunique()}')
print(f'Unique UniqTraits: {ukb_info.uniqTrait.nunique()}')
print(f'Unique PMID: {ukb_info.PMID.nunique()}')
print(f'Unique Consortium: {ukb_info.Consortium.nunique()}')
print(f'Unique Populations: {ukb_info.Population.nunique()}')

Unique domains: 24
Unique chapters: 42
Unique subchapters: 116
Unique traits: 961
Unique UniqTraits: 865
Unique PMID: 159
Unique Consortium: 5
Unique Populations: 2


In [38]:
# remove any with missing N
ukb_info = ukb_info[~ukb_info.N.isna()]
#restrict to those with at least 3 hits
ukb_info = ukb_info[ukb_info.Nhits >= 3]
# remove any with missing Genome information
ukb_info = ukb_info[~ukb_info.Genome.isna()]

In [39]:
def extract_phecode(s):
    if isinstance(s, str):
        parts = [x for x in s.split('. ')]
        for p in parts:
            if 'UKB phenotype field' in p:
                return p.split(': ')[-1].split(' (')[0]
        else:
            return np.nan
    else:
        return np.nan

In [40]:
# extract ukb_phecodes where available in notes
ukb_info['phecode'] = ukb_info.Note.apply(lambda x: extract_phecode(x))

In [41]:
ukb_info['phecode'].value_counts()

phecode
48       1
3849     1
4080     1
4290     1
4291     1
        ..
1628     1
1687     1
1697     1
1707     1
20525    1
Name: count, Length: 214, dtype: int64

Notes field: (need to figure out how to extract)

- 'Population prevalence:'
- 'Case-control GWAS.'
- 'Factor score GWAS.'
- 'Quantitative phenotype'
- 'Dichotomized phetnotype'

In [43]:
# percent missing values
(info.isna().sum()/len(info)).sort_values(ascending=False) * 100

Consortium          90.664424
SNPh2_l_se          87.510513
SNPh2_l             87.510513
Ncase               84.377628
Ncontrol            84.377628
SNPh2               52.460050
Intercept           52.460050
Chi2                52.460050
LambdaGC            52.460050
SNPh2_z             52.460050
SNPh2_se            52.460050
Note                 9.188394
Website              1.619008
Genome               0.315391
DateLastModified     0.000000
DateAdded            0.000000
PMID                 0.000000
Nhits                0.000000
Nsnps                0.000000
Year                 0.000000
N                    0.000000
Population           0.000000
uniqTrait            0.000000
Trait                0.000000
SubchapterLevel      0.000000
ChapterLevel         0.000000
Domain               0.000000
File                 0.000000
ga_idx               0.000000
dtype: float64

In [44]:
ukb_info['Trait'] = ukb_info['Trait'].apply(lambda x: x.lower())
ukb_info['uniqTrait'] = ukb_info['uniqTrait'].apply(lambda x: x.lower())

## Genebass

In [45]:
phenos = pd.read_csv(os.path.join(datadir,'../GeneBASS/', 'genebass_pheno_df_full.txt'), sep='\t', index_col=0)
phenos['description'] = phenos['description'].apply(lambda z: z.lower() if isinstance(z, str) else np.nan)
phenos['coding_description'] = phenos['coding_description'].apply(lambda z: z.lower() if isinstance(z, str) else np.nan)
phenos['n_controls'] = phenos['n_controls'].fillna(0)
phenos['N'] = phenos['n_cases'] + phenos['n_controls']
phenos['gb_idx'] = [x for x in range(len(phenos))]

In [46]:
len(phenos)

4529

In [47]:
phenos = phenos.dropna(subset='N')

In [48]:
phenos=phenos.sort_values(by='N', ascending=False)

In [49]:
matches1 = phenos.merge(info, left_on='description', right_on='Trait', how='inner')
len(matches1)

0

In [50]:
matched_ids = list(matches1.gb_idx.values)

In [51]:
matches2 = phenos[~phenos.gb_idx.isin(matched_ids)].merge(info, left_on='description', right_on='uniqTrait', how='inner')
len(matches2)

0

In [52]:
matched_ids += list(matches2.gb_idx.values)

In [53]:
matches3 = phenos[~phenos.gb_idx.isin(matched_ids)].merge(info, left_on='coding_description', right_on='Trait', how='inner')
len(matches3)

0

In [54]:
matched_ids += list(matches3.gb_idx.values)

In [55]:
matches4 = phenos[~phenos.gb_idx.isin(matched_ids)].merge(info, left_on='coding_description', right_on='uniqTrait', how='inner')
len(matches4)

0

In [56]:
all_matches = pd.concat([matches1, matches2, matches3, matches4])

In [57]:
len(set(matched_ids))

0

In [58]:
phenos[~phenos.gb_idx.isin(matched_ids)]

,n_cases,n_controls,heritability,saige_version,inv_normalized,trait_type,phenocode,pheno_sex,coding,modifier,n_cases_defined,n_cases_both_sexes,n_cases_females,n_cases_males,description,description_more,coding_description,category,N,gb_idx
2264,30511,364330.0,0.012857,SAIGE_0.44.5,False,categorical,41210,both_sexes,Z274,NaN,37861,39028,22428,15433,operative procedures - secondary opcs4,This field is a summary of the secondary opera...,z27.4 duodenum,Health-related outcomes > Hospital inpatient >...,394841.0,2264
2084,1111,393730.0,0.077249,SAIGE_0.44.5,False,categorical,41210,both_sexes,X851,NaN,1368,1422,891,477,operative procedures - secondary opcs4,This field is a summary of the secondary opera...,x85.1 torsion dystonias and other involuntary ...,Health-related outcomes > Hospital inpatient >...,394841.0,2084
2075,163,394678.0,0.122562,SAIGE_0.44.5,False,categorical,41210,both_sexes,X713,NaN,212,219,162,50,operative procedures - secondary opcs4,This field is a summary of the secondary opera...,x71.3 procurement of drugs for chemotherapy fo...,Health-related outcomes > Hospital inpatient >...,394841.0,2075
2076,178,394663.0,0.000000,SAIGE_0.44.5,False,categorical,41210,both_sexes,X715,NaN,234,245,135,99,operative procedures - secondary opcs4,This field is a summary of the secondary opera...,x71.5 procurement of drugs for chemotherapy fo...,Health-related outcomes > Hospital inpatient >...,394841.0,2076
2077,3360,391481.0,0.034723,SAIGE_0.44.5,False,categorical,41210,both_sexes,X721,NaN,4092,4278,2344,1748,operative procedures - secondary opcs4,This field is a summary of the secondary opera...,x72.1 delivery of complex chemotherapy for neo...,Health-related outcomes > Hospital inpatient >...,394841.0,2077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,312,3992.0,0.000000,SAIGE_0.44.5,False,categorical,IBDColectomy_custom,both_sexes,NaN,custom,361,372,159,202,definition of colectomy related to ibd,Gastrointestinal,NaN,Autoimmune/inflammatory,4304.0,2531
879,1273,1620.0,0.024376,SAIGE_0.44.5,False,categorical,20415,both_sexes,NaN,NaN,1502,1536,660,842,ongoing addiction to alcohol,"Question asked: ""Is this addiction ongoing?"" ...",NaN,Online follow-up > Mental health > Addictions,2893.0,879
877,786,2016.0,0.008833,SAIGE_0.44.5,False,categorical,20404,both_sexes,NaN,NaN,926,946,308,618,ever physically dependent on alcohol,"Question asked: ""Have you been physically depe...",NaN,Online follow-up > Mental health > Addictions,2802.0,877
869,535,604.0,0.031158,SAIGE_0.44.5,False,categorical,20122,both_sexes,NaN,NaN,772,807,365,407,bipolar disorder status,This derived data field has come from Professo...,NaN,UK Biobank Assessment Centre > Touchscreen > P...,1139.0,869


## GWAS Catalog study info

In [59]:
import re
def extract_sample_size(s):
    if isinstance(s, str):
        # Find all occurrences of numbers which may include commas.
        matches = re.findall(r'\d[\d,]*', s)
        # Remove commas and convert each match to an integer, then sum them up.
        return sum(int(num.replace(',', '')) for num in matches)
    else:
        return np.nan

In [60]:
cat_info = pd.read_csv(os.path.join(datadir, '../GWASCatalog/study_info.v1.0.3.1_Jan29_2025.txt'), sep='\t')
cat_info['cat_idx'] = [x for x in range(len(cat_info))]

In [61]:
genotyping_techs = ['Genome-wide genotyping array', 'Genome-wide sequencing [NA]', 'Genome-wide sequencing']

In [62]:
ukb_cat = cat_info[cat_info.COHORT.isin(['UKB', 'UKBB'])]

In [63]:
ukb_cat = ukb_cat[ukb_cat['GENOTYPING TECHNOLOGY'].isin(genotyping_techs)]
ukb_cat = ukb_cat[ukb_cat['BACKGROUND TRAIT'].isna()]
ukb_cat['N'] = ukb_cat['INITIAL SAMPLE SIZE'].apply(lambda z: extract_sample_size(z))

In [64]:
ukb_cat = ukb_cat.dropna(subset='N')

In [739]:
ukb_cat = ukb_cat[ukb_cat['ASSOCIATION COUNT'] >=3]

In [740]:
ukb_cat = ukb_cat.sort_values(by='N', ascending=False)

In [741]:
ukb_pmid= set([str(x) for x in ukb_cat['PUBMED ID'].unique()])
atlas_pmid = set(ukb_info.PMID.unique())
len(ukb_pmid.intersection(atlas_pmid))

0

## NO Overlap between UKB studies in Catalog vs Atlas....

In [742]:
ukb_cat['DISEASE/TRAIT'] = ukb_cat['DISEASE/TRAIT'].apply(lambda z: z.lower())
ukb_cat['MAPPED_TRAIT'] = ukb_cat['MAPPED_TRAIT'].apply(lambda z: z.lower() if isinstance(z, str) else np.nan)

In [743]:
cat1 = ukb_cat.merge(info, left_on='DISEASE/TRAIT', right_on='Trait', how='inner')
len(cat1)

0

In [392]:
cat_ids = list(cat1.cat_idx.values)

In [393]:
cat2 = ukb_cat.merge(info, left_on='MAPPED_TRAIT', right_on='Trait', how='inner')
len(cat2)

341

In [394]:
cat_ids += list(cat2.cat_idx.values)

In [395]:
all_cat = pd.concat([cat1, cat2])

In [396]:
a = set(all_cat.Trait.values)

In [397]:
b = set(all_matches.Trait.values)

In [398]:
c = a.intersection(b)
d = a.union(b)

In [399]:
sum(ukb_cat[ukb_cat.cat_idx.isin(cat_ids)]['ASSOCIATION COUNT'] > 5)

124

### Test SBERT

In [73]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [255]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def compute_bidirectional_matches(embeddings1, embeddings2, series1, series2):
    """
    Computes the best matches between two pandas Series of text using SBERT.
    
    For each entry in series1, it finds the best matching entry in series2 and vice versa.
    The results are combined into a single DataFrame with a 'Direction' column.
    
    Parameters:
        series1 (pd.Series): A pandas Series with text entries. The Series index is preserved.
        series2 (pd.Series): A pandas Series with text entries. The Series index is preserved.
        model_name (str): The name of the SentenceTransformer model to use.
        batch_size (int): Batch size for encoding.
        show_progress_bar (bool): Whether to show the progress bar during encoding.
    
    Returns:
        pd.DataFrame: A DataFrame containing the results from both matching directions.
    """
    # Convert Series to lists (order is maintained)
    list1 = series1.tolist()
    list2 = series2.tolist()
    
    # Compute the cosine similarity matrix between embeddings from series1 and series2
    cos_sim_matrix = cosine_similarity(embeddings1, embeddings2)
    
    # -------------------------------
    # Matches: Series1 -> Series2
    # -------------------------------
    best_match_indices_1_to_2 = np.argmax(cos_sim_matrix, axis=1)
    best_match_scores_1_to_2 = np.max(cos_sim_matrix, axis=1)
    
    # Create a DataFrame for matches from series1 to series2 with standardized column names
    df1_to_2 = pd.DataFrame({
        "Direction": "Series1_to_Series2",
        "Query_Index": series1.index,
        "Query_Text": list1,
        "Match_Index": [series2.index[i] for i in best_match_indices_1_to_2],
        "Match_Text": [list2[i] for i in best_match_indices_1_to_2],
        "Cosine_Similarity": best_match_scores_1_to_2
    })
    
    # -------------------------------
    # Matches: Series2 -> Series1
    # -------------------------------
    best_match_indices_2_to_1 = np.argmax(cos_sim_matrix, axis=0)
    best_match_scores_2_to_1 = np.max(cos_sim_matrix, axis=0)
    
    # Create a DataFrame for matches from series2 to series1 with standardized column names
    df2_to_1 = pd.DataFrame({
        "Direction": "Series2_to_Series1",
        "Query_Index": series2.index,
        "Query_Text": list2,
        "Match_Index": [series1.index[i] for i in best_match_indices_2_to_1],
        "Match_Text": [list1[i] for i in best_match_indices_2_to_1],
        "Cosine_Similarity": best_match_scores_2_to_1
    })
    
    # Combine both DataFrames into one for downstream analysis.
    combined_df = pd.concat([df1_to_2, df2_to_1], ignore_index=True)
    
    return combined_df.sort_values(by='Cosine_Similarity', ascending=False)

def extract_mutual_matches(bidirectional_df):

 # Split the bidirectional DataFrame by direction.
    df1_to_2 = bidirectional_df[bidirectional_df['Direction'] == 'Series1_to_Series2'].copy()
    df2_to_1 = bidirectional_df[bidirectional_df['Direction'] == 'Series2_to_Series1'].copy()

    # Rename columns for df1_to_2.
    df1_to_2.rename(columns={
        "Query_Index": "Series1_Index",
        "Query_Text": "Series1_Text",
        "Match_Index": "Series2_Index",
        "Match_Text": "Series2_Text",
        "Cosine_Similarity": "Cosine_Similarity_1"
    }, inplace=True)

    # Rename columns for df2_to_1.
    df2_to_1.rename(columns={
        "Query_Index": "Series2_Index",
        "Query_Text": "Series2_Text",
        "Match_Index": "Series1_Index",
        "Match_Text": "Series1_Text",
        "Cosine_Similarity": "Cosine_Similarity_2"
    }, inplace=True)

    # Merge the two DataFrames on Series1_Index and Series2_Index.
    mutual = pd.merge(df1_to_2, df2_to_1, on=["Series1_Index", "Series2_Index"], suffixes=('_1', '_2'))
    
    # Compute the average cosine similarity.
    mutual["Average_Cosine_Similarity"] = (mutual["Cosine_Similarity_1"] + mutual["Cosine_Similarity_2"]) / 2.0

    # Select columns for output. Here we choose the text columns from the first direction (suffix _1).
    result = mutual[[
        "Series1_Index", "Series1_Text_1", "Series2_Index", "Series2_Text_1", "Average_Cosine_Similarity"
    ]].copy()
    
    # Rename the suffixed text columns to standard names.
    result.rename(columns={"Series1_Text_1": "Series1_Text", "Series2_Text_1": "Series2_Text"}, inplace=True)
    
    return result


In [74]:
# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

## BERT matching of GeneBASS and GWAS Catalog

In [745]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist

def get_all_matches(series1, series2, emb1, emb2, th=0.8):
    """
    Given two pandas Series (series1 and series2) along with their precomputed embeddings (emb1 and emb2),
    compute the cosine similarity between every string from series1 and every string from series2.
    
    Returns a DataFrame with all pairs (one string from each Series) having a cosine similarity >= th.
    Each row includes:
      - Series1_Index, Series1_Text,
      - Series2_Index, Series2_Text,
      - Cosine_Similarity.
    
    Parameters:
      series1 (pd.Series): Series of strings (with index) for list1.
      series2 (pd.Series): Series of strings (with index) for list2.
      emb1 (np.ndarray): Embeddings for the strings in series1 (assumed to correspond in order to series1.index).
      emb2 (np.ndarray): Embeddings for the strings in series2.
      th (float): Cosine similarity threshold (default=0.8).
    
    Returns:
      pd.DataFrame: DataFrame containing all unique pairs with similarity >= th.
    """
    # Compute the cosine similarity matrix between emb1 and emb2.
    sim_matrix = cosine_similarity(emb1, emb2)
    
    pairs = []
    # Loop over rows (series1) and columns (series2) to collect pairs meeting threshold.
    n1, n2 = sim_matrix.shape
    for i in range(n1):
        for j in range(n2):
            sim = sim_matrix[i, j]
            if sim >= th:
                pairs.append({
                    "Series1_Index": series1.index[i],
                    "Series1_Text": series1.iloc[i],
                    "Series2_Index": series2.index[j],
                    "Series2_Text": series2.iloc[j],
                    "Cosine_Similarity": sim
                })
    
    # Create DataFrame of matches.
    matches_df = pd.DataFrame(pairs)
    # (If the same pair could be found twice by any chance, drop duplicates.)
    matches_df.drop_duplicates(inplace=True)
    # Optionally, sort by descending similarity.
    matches_df.sort_values(by="Cosine_Similarity", ascending=False, inplace=True)
    return matches_df


In [746]:
cat_embeddings = model.encode(list(ukb_cat['DISEASE/TRAIT'].values), batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/113 [00:00<?, ?it/s]

In [748]:
desc_counts = phenos.description.value_counts()
use_description = phenos[~phenos.description.isin(desc_counts[desc_counts>3].index.values)]
use_description = use_description[~use_description.description.isna()]
use_description.shape

(2058, 20)

In [749]:
bass_embeddings1 = model.encode(use_description.description.values, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

In [758]:
cat_desc = get_all_matches(ukb_cat.set_index('cat_idx')['DISEASE/TRAIT'], 
                          use_description.set_index('gb_idx').description, 
                          cat_embeddings, bass_embeddings1, th=0.7)

In [756]:
use_coding = phenos[~phenos.coding_description.isna()]
bass_embeddings2 = model.encode(use_coding.coding_description.values, batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/77 [00:00<?, ?it/s]

In [759]:
cat_coding = get_all_matches(ukb_cat.set_index('cat_idx')['DISEASE/TRAIT'], 
                          use_coding.set_index('gb_idx').coding_description, 
                          cat_embeddings, bass_embeddings2, th=0.7)

In [760]:
cat_coding.tail()

,Series1_Index,Series1_Text,Series2_Index,Series2_Text,Cosine_Similarity
41,65507,depression (time to event),906,depression,0.701610
40,65507,depression (time to event),106,depression,0.701610
116,1943,major depressive disorder,906,depression,0.700348
115,1943,major depressive disorder,106,depression,0.700348
1,13140,rheumatoid arthritis or type 1 diabetes,76,type 1 diabetes,0.700347


In [761]:
cat_matches = pd.concat([cat_desc, cat_coding])

#### Export UKB Common trait info

In [762]:
out_cat = ukb_cat[ukb_cat.cat_idx.isin(cat_matches.Series1_Index.values)]

In [763]:
out_cat = out_cat.loc[:, ('PUBMED ID','DATE', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE','N', 'ASSOCIATION COUNT', 
                               'MAPPED_TRAIT', 'MAPPED_TRAIT_URI','STUDY ACCESSION', 'cat_idx')]

In [767]:
def extract_ancestry(pop):
    if ', ' in pop:
        pop = pop.split(', ')
        outstr = ''
        for x in pop:
            outstr += x.split(' ', 1)[-1]
    else:
        outstr = pop.split(' ', 1)[1]
    return outstr

In [768]:
out_cat['pop'] = [extract_ancestry(x) for x in out_cat['INITIAL SAMPLE SIZE'].values]

In [772]:
pop_map = {'European ancestry casesEuropean ancestry controlsSouth Asian ancestry casesSouth Asian ancestry controlsEast Asian ancestry casesEast Asian ancestry controlsAfrican ancestry casesAfrican ancestry controls': 'Mixed',
       'European ancestry rheumatoid arthritis casesEuropean ancestry type 1 diabetes cases or controlsEuropean ancestry controls': 'EUR',
       'European ancestry casesEuropean ancestry controls': 'EUR',
       'European or unknown ancestry casesEuropean or unknown ancestry controls': 'EUR',
       'European ancestry individuals': 'EUR',
       'European ancestry casesEuropean ancestry controlscasescontrols': 'EUR',
       'European ancestry casesEuropean ancestry controlsSouth Asianor unknown ancestry casesSouth Asianor unknown ancestry controls':'EUR, SAS',
       'European or unknown ancestry individuals': 'EUR',
       'to 460,858 European or unknown ancestry individuals': 'EUR',
       'European ancestry rheumatoid arthritis casesEuropean ancestry inflammatory bowel disease casesleast 447,182 European ancestry controls': 'EUR',
       'European ancestry rheumatoid arthritis casesEuropean ancestry multiple sclerosis casesleast 447,182 European ancestry controls': 'EUR',
       'White-British ancestry casesWhite-British ancestry controls': 'EUR',
       'British ancestry individualsAsian or Asian British individualsBlack or Black British individualsChinese ancestry individuals':'Mixed',
       'British individuals': 'EUR',
       'British ancestry casesBritish ancestry controls': 'EUR', 'individuals': 'EUR',
       'British ancestry individuals': 'EUR', 'casescontrols': 'EUR',
       'European ancestry individualsAfrican ancestry individualsSouth Asian ancestry individuals':'Mixed',
       'African ancestry individualsEast Asian ancestry individualsEuropean ancestry individualsSouth Asian ancestry individuals':'Mixed',
       'White British ancestry casesWhite British ancestry controls': 'EUR',
       'European ancestry female casesEuropean ancestry female controlsfemale casesfemale controls': 'EUR',
       'European ancestry female casesEuropean ancestry female controlsSouth Asianor unknown ancestry female casesSouth Asianor unknown ancestry female controls':'EUR, SAS',
       'European ancestry female casesEuropean ancestry female controls': 'EUR',
       'menwomen': 'EUR',
       'European ancestry male casesEuropean ancestry male controlsSouth Asianor unknown ancestry male casesSouth Asianor unknown ancestry male controls':'EUR, SAS',
       'European ancestry male casesEuropean ancestry male controlsmale casesmale controls': 'EUR',
       'European ancestry male casesEuropean ancestry male controls': 'EUR',
       'women': 'EUR',
       'African ancestry individualsEuropean ancestry individuals':'EUR, AFR', 'men':'EUR',
       'British ancestry women': 'EUR', 'British ancestry males': 'EUR',
       'British ancestry females': 'EUR', 'British ancestry men': 'EUR',
       'European ancestry women': 'EUR',
       'British ancestry female casesBritish ancestry female controls': 'EUR',
       'European ancestry men': 'EUR',
       'British ancestry male casesBritish ancestry male controls': 'EUR',
       'African ancestry casesAfrican ancestry controlsEuropean ancestry casesEuropean ancestry controlsSouth Asian ancestry casesSouth Asian ancestry controlscasescontrols':'Mixed',
       'British ancestry post-menopausal women': 'EUR',
       'African ancestry female casesAfrican ancestry female controlsEuropean ancestry female casesEuropean ancestry female controlsSouth Asian ancestry female casesSouth Asian ancestry female controlsfemale casesfemale controls':'Mixed',
       'European ancestry females': 'EUR',
       'African ancestry male casesAfrican ancestry male controlsEuropean ancestry male casesEuropean ancestry male controlsSouth Asian ancestry male casesSouth Asian ancestry male controlsmale casesmale controls':'Mixed',
       'European ancestry males': 'EUR',
       'European ancestry individualsAsian ancestry individualsBlack individualsMixed ancestry individualsindividuals':'Mixed',
       'European ancestryAsian ancestryAsian ancestryancestryancestry individuals':'EUR, EAS',
       'European ancestry individualsAfrican ancestry individualsSouth Asian ancestry individualsEast Asian ancestry individualsindividuals':'Mixed',
       'British ancestry pre-menopausal women': 'EUR',
       'EuropeanIndianCaribbeanancestry cases EuropeanIndianCaribbeanancestry controls':'Mixed',
       'South Asianor unknown ancestry casesSouth Asianor unknown ancestry controls':'SAS',
       'European ancestry menAsian ancestry menBlack menMixed ancestry menmen':'Mixed',
       'European ancestryAsian ancestryAsian ancestryancestryancestry females':'EUR, EAS',
       'European ancestryAsian ancestryAsian ancestryancestryancestry males':'EUR, EAS',
       'EuropeanIndianCaribbeanancestry casesEuropeanIndianCaribbeanancestry ancestry controls':'Mixed',
       'South Asian ancestry individuals':'SAS',
       'South Asian ancestry casesSouth Asian ancestry controls':'SAS',
       'African American or Afro-Caribbean individuals':'AFR',
       'African ancestry individuals': 'AFR',
       'White British ancestry male individuals': 'EUR',
       'African ancestry casesAfrican ancestry controls': 'AFR'}

In [773]:
out_cat['Ancestry'] = out_cat['pop'].map(pop_map)


In [777]:
ukb_cat_out = out_cat.loc[:, ('PUBMED ID','DATE', 'DISEASE/TRAIT', 'N', 'Ancestry',  'ASSOCIATION COUNT', 
                               'MAPPED_TRAIT', 'MAPPED_TRAIT_URI','STUDY ACCESSION', 'cat_idx')]
ukb_cat_out['StudyC'] = 'GC_idx'+ukb_cat_out.cat_idx.astype(str) + '_' + ukb_cat_out['STUDY ACCESSION']

In [778]:
ukb_cat_out.to_csv('/cellar/users/snwright/Data/RareCommon/GWASCatalog/UKB_GWASCat_Genebass_matched_traits.tsv', sep='\t', index=False)

In [779]:
sum(ukb_cat_out['ASSOCIATION COUNT']>3)

545

### Export UKB Rare trait info

In [780]:
mutual_gb = phenos[phenos.gb_idx.isin(cat_matches.Series2_Index.values)]

In [781]:
mutual_gb = mutual_gb.merge(cat_matches.loc[:,('Series2_Index', 'Series2_Text')], left_on='gb_idx', right_on='Series2_Index')

In [782]:
mutual_gb = mutual_gb.loc[:, ('n_cases', 'n_controls', 'heritability','trait_type', 'N','phenocode', 'coding', 'Series2_Text', 'gb_idx')]

In [784]:
mutual_gb = mutual_gb[mutual_gb.trait_type!='icd_first_occurrence']

In [786]:
mutual_gb['Classification']= mutual_gb['trait_type'].map({'categorical':'Population CC', 'continuous':'Population Q', 
                                                          'icd10':'Population CC', 'icd_first_occurrence':'Exclude'})

In [787]:
mutual_gb_out = mutual_gb.loc[:, ('gb_idx', 'Series2_Text', 'N', 'Classification', 'phenocode', 'coding', 'heritability')]

In [789]:
mutual_gb_out['StudyR'] = 'GB_UKB_idx' + mutual_gb_out.gb_idx.astype(str) + '_phe' + mutual_gb_out.phenocode.astype(str)

In [790]:
mutual_gb_out.to_csv('/cellar/users/snwright/Data/RareCommon/GeneBASS/UKB_Genebass_GWASCat_matched_traits.tsv', sep='\t', index=False)

### Export matches

In [792]:
cat_matches.columns= ['cat_idx', 'DISEASE/TRAIT', 'gb_idx', 'description', 'Cosine']

In [793]:
cat_matches = cat_matches.merge(mutual_gb_out.loc[:, ('gb_idx', 'StudyR')], on='gb_idx').merge(ukb_cat_out.loc[:, ('cat_idx', 'StudyC')], on='cat_idx')
cat_matches.to_csv('/cellar/users/snwright/Data/RareCommon/inputs/UKB/trait_matches_genebass_gwascat_Mar12.txt', sep='\t', index=False)

## BERT matching of Genebass and GwasAtlas

In [65]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist

def get_all_matches(series1, series2, emb1, emb2, th=0.8):
    """
    Given two pandas Series (series1 and series2) along with their precomputed embeddings (emb1 and emb2),
    compute the cosine similarity between every string from series1 and every string from series2.
    
    Returns a DataFrame with all pairs (one string from each Series) having a cosine similarity >= th.
    Each row includes:
      - Series1_Index, Series1_Text,
      - Series2_Index, Series2_Text,
      - Cosine_Similarity.
    
    Parameters:
      series1 (pd.Series): Series of strings (with index) for list1.
      series2 (pd.Series): Series of strings (with index) for list2.
      emb1 (np.ndarray): Embeddings for the strings in series1 (assumed to correspond in order to series1.index).
      emb2 (np.ndarray): Embeddings for the strings in series2.
      th (float): Cosine similarity threshold (default=0.8).
    
    Returns:
      pd.DataFrame: DataFrame containing all unique pairs with similarity >= th.
    """
    # Compute the cosine similarity matrix between emb1 and emb2.
    sim_matrix = cosine_similarity(emb1, emb2)
    
    pairs = []
    # Loop over rows (series1) and columns (series2) to collect pairs meeting threshold.
    n1, n2 = sim_matrix.shape
    for i in range(n1):
        for j in range(n2):
            sim = sim_matrix[i, j]
            if sim >= th:
                pairs.append({
                    "Series1_Index": series1.index[i],
                    "Series1_Text": series1.iloc[i],
                    "Series2_Index": series2.index[j],
                    "Series2_Text": series2.iloc[j],
                    "Cosine_Similarity": sim
                })
    
    # Create DataFrame of matches.
    matches_df = pd.DataFrame(pairs)
    # (If the same pair could be found twice by any chance, drop duplicates.)
    matches_df.drop_duplicates(inplace=True)
    # Optionally, sort by descending similarity.
    matches_df.sort_values(by="Cosine_Similarity", ascending=False, inplace=True)
    return matches_df


In [66]:
ukb_info.head(1)

,PMID,Year,File,Website,Consortium,Domain,ChapterLevel,SubchapterLevel,Trait,uniqTrait,...,SNPh2_l,SNPh2_l_se,LambdaGC,Chi2,Intercept,Note,DateAdded,DateLastModified,ga_idx,phecode
id,,,,,,,,,,,,,,,,,,,,,
41,27046643,2016,http://www.psy.ed.ac.uk/ccace/downloads/Davies...,http://www.ccace.ed.ac.uk/node/335,CCACE,Environment,Major Life Areas,Education,educational attainment,educational attainment,...,NaN,NaN,1.3068,1.3685,1.0342,UK Biobank cohort. The number of risk loci rep...,2017-07-04,2017-07-04,40,NaN


In [67]:
phenos.phenocode.value_counts()

phenocode
41210    787
41200    735
20003    422
20002    235
20004    182
        ... 
23104      1
23102      1
23098      1
1687       1
20504      1
Name: count, Length: 2097, dtype: int64

In [68]:
# first direct matches based on phecodes - do not use 'coding' from genebass will give incorrect matches
phe_match = phenos.merge(ukb_info, left_on='phenocode', right_on='phecode', how='inner')
phe_match.shape

(69, 50)

In [69]:
ga_match_ids = list(phe_match.ga_idx.values)

#### Descriptions

In [70]:
# get the traits where we will use the description column
desc_counts = phenos.description.value_counts()
use_description = phenos[~phenos.description.isin(desc_counts[desc_counts>3].index.values)]
use_description = use_description[~use_description.description.isna()]
use_description.shape

(2058, 20)

In [75]:
# get embeddings
ga_embeddings = model.encode(list(ukb_info[~ukb_info.ga_idx.isin(ga_match_ids)]['Trait'].values), batch_size=32, show_progress_bar=True)
bass_embeddings1 = model.encode(use_description.description.values, batch_size=32, show_progress_bar=True)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/65 [00:00<?, ?it/s]

In [76]:
ga_desc = get_all_matches(ukb_info[~ukb_info.ga_idx.isin(ga_match_ids)].set_index('ga_idx').Trait, 
                          use_description.set_index('gb_idx').description, 
                          ga_embeddings, bass_embeddings1, th=0.7)

#### Coding descriptions

In [77]:
use_coding = phenos[(~phenos.coding_description.isna()) & (~phenos.gb_idx.isin(use_description.gb_idx.values))]
bass_embeddings2 = model.encode(use_coding.coding_description.values, batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/76 [00:00<?, ?it/s]

In [78]:
ga_coding = get_all_matches(ukb_info[~ukb_info.ga_idx.isin(ga_match_ids)].set_index('ga_idx').Trait, 
                          use_coding.set_index('gb_idx').coding_description, 
                          ga_embeddings, bass_embeddings2, th=0.7)

#### Combine dfs and identify groups

In [89]:
ga_all = pd.concat([ga_desc, ga_coding])

In [88]:
ga_all.head()

,ga_idx,Trait,gb_idx,description,Cosine
34,4174,heel bone mineral density,3758,heel bone mineral density,1.000000
8,3238,exposure to tobacco smoke outside home,2618,exposure to tobacco smoke at home,0.947096
49,4293,chronotype,3760,chronotype (quantitative),0.903574
1,1140,chronotype,3760,chronotype (quantitative),0.903574
2,1141,sleep duration,3789,sleep duration (quantitative),0.896718


In [80]:
series_gb = pd.concat([use_description.set_index('gb_idx').description, use_coding.set_index('gb_idx').coding_description])
len(series_gb)

4490

In [81]:
bass_emb = np.concatenate([bass_embeddings1, bass_embeddings2])
bass_emb.shape

(4490, 384)

### Export

In [90]:
ga_all.columns = ['ga_idx', 'Trait', 'gb_idx', 'description', 'Cosine']

In [91]:
phe_combined = phe_match.loc[:, ('ga_idx', 'Trait', 'gb_idx', 'description')]
phe_combined['Cosine'] = 1.0

In [94]:
all_matches = pd.concat([ga_all, phe_combined])

In [101]:
ukb_info_out = ukb_info[ukb_info.ga_idx.isin(all_matches.ga_idx.values)]

In [102]:
ukb_info_out.drop_duplicates(subset=['PMID', 'Trait', 'Population']).shape

(166, 30)

In [103]:
ukb_info_out = ukb_info_out.assign(Population= ukb_info_out.Population.map({'UKB1 (EUR)':'UKB1', 'UKB2 (EUR)':'UKB2', 
                                                             'UKB2 (EUR meta)':'UKB2meta', 'UKB1 (EUR meta)':'UKB1meta'}))

In [104]:
ukb_info_out.Population.unique()

array(['UKB1', 'UKB2'], dtype=object)

In [105]:
ukb_info_out['StudyC'] = 'GA_idx'+ukb_info_out.ga_idx.astype(str) +'_'+ ukb_info_out.Population + '_' + ukb_info_out.PMID.astype(str)

In [106]:
ukb_info_out.drop(columns=['File', 'Website', 'Consortium', 'DateAdded', 'DateLastModified']).to_csv('/cellar/users/snwright/Data/RareCommon/GWASAtlas/UKB_Atlas_Genebass_matched_traits.tsv', sep='\t', index=False)

In [107]:
phenos_out1 = phenos[phenos.gb_idx.isin(all_matches.gb_idx.values)]

In [109]:
phenos_out1['StudyR'] = 'GB_UKB_idx' + phenos_out1.gb_idx.astype(str) + '_phe' + phenos_out1.phenocode.astype(str)

/tmp/ipykernel_3352879/2085730337.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  phenos_out1['StudyR'] = 'GB_UKB_idx' + phenos_out1.gb_idx.astype(str) + '_phe' + phenos_out1.phenocode.astype(str)


In [110]:
phenos_out1.columns

Index(['n_cases', 'n_controls', 'heritability', 'saige_version',
       'inv_normalized', 'trait_type', 'phenocode', 'pheno_sex', 'coding',
       'modifier', 'n_cases_defined', 'n_cases_both_sexes', 'n_cases_females',
       'n_cases_males', 'description', 'description_more',
       'coding_description', 'category', 'N', 'gb_idx', 'StudyR'],
      dtype='object')

In [111]:
phenos_out1.to_csv('/cellar/users/snwright/Data/RareCommon/GeneBASS/UKB_Atlas_matched_trait_info.txt', sep='\t')

In [116]:
all_matches = all_matches.merge(phenos_out1.loc[:, ('gb_idx', 'StudyR')], on='gb_idx').merge(ukb_info_out.loc[:, ('ga_idx', 'StudyC')], on='ga_idx')
all_matches.to_csv('/cellar/users/snwright/Data/RareCommon/inputs/UKB/trait_matches_genebass_atlas_Mar21.txt', sep='\t', index=False)

In [117]:
all_matches.to_csv('/cellar/users/snwright/Data/RareCommon/inputs/UKB/trait_matches_genebass_atlas_Mar21.txt', sep='\t', index=False)

## RAVAR

In [797]:
ravar_info = pd.read_csv('/cellar/users/snwright/Data/RareCommon/Annotations/rare_info/cleaned_info_gwascat_matches.tsv', sep='\t')
ravar_ukb = ravar_info[ravar_info.COHORT=='UKB']
ukb_pmid = list(ravar_ukb.PMID.unique()) + [33226994,33574263,34626176,36050321,36297015,36311265,36450729]

In [809]:
ravar_genes = pd.read_csv(os.path.join(datadir, '../RAVAR' ,'gene_fulltable_06112024.txt.entrez'),sep='\t', 
                            usecols=['Gene Symbol', 'Ensembl ID', 'Gene Type', 'Entrez', 'CHR', 'Location', 'Reported Trait', 'Trait Label', 'Trait Ontology id', 'EFO synonym', 'P-value', 'PMID'])
#replace '−' with '-'
ravar_genes['P-value'] = ravar_genes['P-value'].apply(lambda x: float(x.replace('−','-')) if type(x) == str else float(x))
ravar_genes['TRAIT_CODE'] = ravar_genes['Trait Ontology id'].apply(lambda x: x.replace(":", "_") if type(x) == str else x)
ravar_genes['logp'] = -1 * np.log10(ravar_genes['P-value'] + 1e-250)

In [810]:
ukb_assoc = ravar_genes[ravar_genes.PMID.isin(ukb_pmid)]
gene_counts = ukb_assoc.groupby(['PMID', 'TRAIT_CODE', 'Reported Trait']).Entrez.nunique().reset_index().rename(columns={'Entrez':'geneCount'})
ukb_assoc = ukb_assoc.merge(gene_counts, on=['PMID', 'TRAIT_CODE', 'Reported Trait'])

In [812]:
ukb_assoc = ukb_assoc[ukb_assoc.geneCount>= 3]

In [818]:
ukb_assoc = ukb_assoc.drop_duplicates(['PMID', 'TRAIT_CODE', 'Reported Trait'])
ukb_assoc['ravar_idx'] = [x for x in range(len(ukb_assoc))]

In [820]:
ravar_emb=model.encode(ukb_assoc['Reported Trait'].values, batch_size=32)

### RAVAR + GWAS Cat

In [902]:
match1 = get_all_matches(ukb_cat.set_index('cat_idx')['DISEASE/TRAIT'], 
                ukb_assoc.set_index('ravar_idx')['Reported Trait'], 
                cat_embeddings, ravar_emb, th=0.8)

In [903]:
match1.head()

,Series1_Index,Series1_Text,Series2_Index,Series2_Text,Cosine_Similarity
5,9491,angina,1211,Angina,1.0
428,106787,hernia,603,Hernia,1.0
208,33656,mean reticulocyte volume,1497,Mean reticulocyte volume,1.0
130,29325,mean reticulocyte volume,1497,Mean reticulocyte volume,1.0
152,35295,triglycerides,20,Triglycerides,1.0


#### Export RAVAR + gwasCAT

In [857]:
out_cat2 = ukb_cat[ukb_cat.cat_idx.isin(match1.Series1_Index.values)]

out_cat2 = out_cat2.loc[:, ('PUBMED ID','DATE', 'DISEASE/TRAIT', 'INITIAL SAMPLE SIZE','N', 'ASSOCIATION COUNT', 
                               'MAPPED_TRAIT', 'MAPPED_TRAIT_URI','STUDY ACCESSION', 'cat_idx')]
out_cat2['pop'] = [extract_ancestry(x) for x in out_cat2['INITIAL SAMPLE SIZE'].values]

In [860]:
print([x for x in out_cat2['pop'].unique() if x not in pop_map])
pop_map['white-British ancestry individuals'] = 'EUR'

['white-British ancestry individuals']


In [861]:
out_cat2['Ancestry'] = out_cat2['pop'].map(pop_map)

In [862]:
ukb_cat_out2 = out_cat2.loc[:, ('PUBMED ID','DATE', 'DISEASE/TRAIT', 'N', 'Ancestry',  'ASSOCIATION COUNT', 
                               'MAPPED_TRAIT', 'MAPPED_TRAIT_URI','STUDY ACCESSION', 'cat_idx')]
ukb_cat_out2['StudyC'] = 'GC_idx'+ukb_cat_out2.cat_idx.astype(str) + '_' + ukb_cat_out2['STUDY ACCESSION']

ukb_cat_out2.to_csv('/cellar/users/snwright/Data/RareCommon/GWASCatalog/UKB_GWASCat_RAVAR_matched_traits.tsv', sep='\t', index=False)

In [ ]:
#rare

In [908]:
ravar_info.head()

,Reported Trait,Trait Label,Trait Ontology id,PMID,COHORT,Ancestry,Notes,ExtraNote,index,N,Classification
0,Abnormal findings on diagnostic imaging of oth...,abnormal result of diagnostic imaging,EFO:0009827,34375979,UKB,EUR,NaN,NaN,1,281104.0,Population CC/Q
1,Biochemistry Total Protein measurement,total blood protein measurement,EFO:0004536,34226706,UKB,EUR,NaN,NaN,2,400482.0,Population Q
2,Biochemistry Calcium measurement,calcium measurement,EFO:0004838,34226706,UKB,EUR,NaN,NaN,3,400792.0,Population Q
3,Biochemistry SHBG measurement,sex hormone-binding globulin measurement,EFO:0004696,34226706,UKB,EUR,NaN,NaN,4,397043.0,Population Q
4,Biochemistry Gamma Glutamyl transferase measur...,serum gamma-glutamyl transferase measurement,EFO:0004532,34226706,UKB,EUR,NaN,NaN,5,437651.0,Population Q


In [904]:
ravar_out1 = ukb_assoc[ukb_assoc.ravar_idx.isin(match1.Series2_Index).values].drop(columns=['Gene Symbol', 'Ensembl ID', 
                                                                                           'Gene Type', 
                                                                                           'CHR', 'Location',
                                                                                           'logp','P-value', 'Entrez'])

In [905]:
ravar_out1['StudyR'] = 'UKB_RAVAR_idx' + ravar_out1['PMID'].astype(str) + '_' + ravar_out1['TRAIT_CODE']

In [913]:
ravar_out1 = ravar_out1.merge(ravar_info.loc[:, ('PMID', 'Reported Trait', 'N')], on=['PMID', 'Reported Trait'],  how='inner')

In [915]:
ravar_out1.to_csv('/cellar/users/snwright/Data/RareCommon/RAVAR/UKB_GWAScat_matched_trait_info.txt', sep='\t', index=False)

In [909]:
ravar_out1.head()

,Reported Trait,Trait Label,Trait Ontology id,EFO synonym,PMID,TRAIT_CODE,geneCount,ravar_idx,StudyR
4,Biochemistry Gamma Glutamyl transferase measur...,serum gamma-glutamyl transferase measurement,EFO:0004532,GGT measurement;Gamma gluatamyl transferase le...,34226706.0,EFO_0004532,33,4,UKB_RAVAR_idx34226706.0_EFO_0004532
6,Biochemistry Total Bilirubin measurement,bilirubin measurement,EFO:0004570,bilirubin levels,34226706.0,EFO_0004570,23,6,UKB_RAVAR_idx34226706.0_EFO_0004570
7,Biochemistry Triglycerides,triglyceride,CHEBI:17855,triglycerides;Triglyceride,34226706.0,CHEBI_17855,25,7,UKB_RAVAR_idx34226706.0_CHEBI_17855
8,Biochemistry Apolipoprotein B measurement,apolipoprotein B change measurement,EFO:0020874,NaN,34226706.0,EFO_0020874,28,8,UKB_RAVAR_idx34226706.0_EFO_0020874
11,Apolipoprotein b (30640) measurement,apolipoprotein B change measurement,EFO:0020874,NaN,34662886.0,EFO_0020874,18,11,UKB_RAVAR_idx34662886.0_EFO_0020874


In [892]:
# matches

In [906]:
match1.columns = ['cat_idx', 'DISEASE/TRAIT', 'ravar_idx', 'Reported Trait', 'Cosine']

In [907]:
match1 = match1.merge(ravar_out1.loc[:, ('ravar_idx', 'StudyR')], on='ravar_idx').merge(ukb_cat_out2.loc[:, ('cat_idx', 'StudyC')], on='cat_idx')
match1.to_csv('/cellar/users/snwright/Data/RareCommon/inputs/UKB/trait_matches_ravar_gwascat_Mar12.txt', sep='\t', index=False)

### RAVAR + GWAS Atlas

In [834]:
ga_embeddings2 = model.encode(list(ukb_info['Trait'].values), batch_size=32, show_progress_bar=True)


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [842]:
match2 = get_all_matches(ukb_info.set_index('ga_idx')['Trait'], 
                ukb_assoc.set_index('ravar_idx')['Reported Trait'], 
                ga_embeddings2, ravar_emb, th=0.8)

#### Export RAVAR + GWAS Atlas

In [885]:
ukb_info_out2 = ukb_info[ukb_info.ga_idx.isin(match2.Series1_Index.values)]

In [888]:
ukb_info_out2 = ukb_info_out2.assign(Population= ukb_info_out2.Population.map({'UKB1 (EUR)':'UKB1', 'UKB2 (EUR)':'UK2', 
                                                             'UKB2 (EUR meta)':'UKB2meta', 'UKB1 (EUR meta)':'UKB1meta'}))

In [890]:
ukb_info_out2['StudyC'] = 'GA_idx'+ukb_info_out2.ga_idx.astype(str) +'_'+ ukb_info_out2.Population + '_' + ukb_info_out2.PMID.astype(str)

In [891]:
ukb_info_out2.drop(columns=['File', 'Website', 'Consortium', 'DateAdded', 'DateLastModified']).to_csv('/cellar/users/snwright/Data/RareCommon/GWASAtlas/UKB_Atlas_RAVAR_matched_traits.tsv', sep='\t', index=False)

In [873]:
ravar_out = ukb_assoc[ukb_assoc.ravar_idx.isin(match2.Series2_Index).values].drop(columns=['Gene Symbol', 'Ensembl ID', 
                                                                                           'Gene Type', 
                                                                                           'CHR', 'Location',
                                                                                           'logp','P-value', 'Entrez'])

In [878]:
ravar_out['StudyR'] = 'UKB_RAVAR_idx' + ravar_out['PMID'].astype(str) + '_' + ravar_out['TRAIT_CODE']

In [916]:
ravar_out = ravar_out.merge(ravar_info.loc[:, ('PMID', 'Reported Trait', 'N')], on=['PMID', 'Reported Trait'],  how='inner')

In [917]:
ravar_out.to_csv('/cellar/users/snwright/Data/RareCommon/RAVAR/UKB_Atlas_matched_trait_info.txt', sep='\t', index=False)

In [900]:
match2.columns = ['ga_idx', 'Trait', 'ravar_idx', 'Reported Trait', 'Cosine']

In [901]:
match2 = match2.merge(ravar_out.loc[:, ('ravar_idx', 'StudyR')], on='ravar_idx').merge(ukb_info_out2.loc[:, ('ga_idx', 'StudyC')], on='ga_idx')
match2.to_csv('/cellar/users/snwright/Data/RareCommon/inputs/UKB/trait_matches_ravar_atlas_Mar12.txt', sep='\t', index=False)

## OLD code

In [869]:
ravar_ukb.head()

,Reported Trait,Trait Label,Trait Ontology id,PMID,COHORT,Ancestry,Notes,ExtraNote,index,N,Classification
0,Abnormal findings on diagnostic imaging of oth...,abnormal result of diagnostic imaging,EFO:0009827,34375979,UKB,EUR,NaN,NaN,1,281104.0,Population CC/Q
1,Biochemistry Total Protein measurement,total blood protein measurement,EFO:0004536,34226706,UKB,EUR,NaN,NaN,2,400482.0,Population Q
2,Biochemistry Calcium measurement,calcium measurement,EFO:0004838,34226706,UKB,EUR,NaN,NaN,3,400792.0,Population Q
3,Biochemistry SHBG measurement,sex hormone-binding globulin measurement,EFO:0004696,34226706,UKB,EUR,NaN,NaN,4,397043.0,Population Q
4,Biochemistry Gamma Glutamyl transferase measur...,serum gamma-glutamyl transferase measurement,EFO:0004532,34226706,UKB,EUR,NaN,NaN,5,437651.0,Population Q


In [192]:
df[df['Cosine Similarity'] > 0.7].List1.nunique()

829

In [193]:
df[df['Cosine Similarity'] > 0.8]['Best Match in List2'].nunique()

268

In [195]:
df.head(100)

,List1,Best Match in List2,Cosine Similarity
1361,sciatica,sciatica,1.000001
1320,bone disorder,bone disorder,1.000000
112,angina,angina,1.000000
1311,angina,angina,1.000000
20156,rheumatoid arthritis,rheumatoid arthritis,1.000000
...,...,...,...
1297,other neurological problem,other neurological problem,1.000000
16011,osteoarthritis,osteoarthritis,1.000000
1303,joint disorder,joint disorder,1.000000
16010,osteoarthritis,osteoarthritis,1.000000


In [144]:
# Example lists of strings (replace these with your actual data)
list1 = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is fascinating.",
    "Artificial intelligence is transforming the world.",
    "Lung cancer"
]
list2 = [
    "A fast, dark-colored fox leaps above a sleeping canine.",
    "I find AI to be very interesting.",
    "The study of machine intelligence is evolving rapidly.",
    'Self reported illness code or trait: lung cancer'
]

# Load the pre-trained SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode both lists using batch processing for efficiency
embeddings1 = model.encode(list1, batch_size=32, show_progress_bar=True)
embeddings2 = model.encode(list2, batch_size=32, show_progress_bar=True)

# Compute cosine similarity matrix between embeddings1 and embeddings2
# Each entry [i, j] corresponds to the cosine similarity between list1[i] and list2[j]
cos_sim_matrix = cosine_similarity(embeddings1, embeddings2)

# For each string in list1, identify the best matching string in list2
best_match_indices = np.argmax(cos_sim_matrix, axis=1)
best_match_scores = np.max(cos_sim_matrix, axis=1)

# Display the best matches and their cosine similarity scores
for idx, match_idx in enumerate(best_match_indices):
    print(f"List1: '{list1[idx]}'")
    print(f"Best match in List2: '{list2[match_idx]}'")
    print(f"Cosine similarity: {best_match_scores[idx]:.4f}")
    print("-" * 80)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

List1: 'The quick brown fox jumps over the lazy dog.'
Best match in List2: 'A fast, dark-colored fox leaps above a sleeping canine.'
Cosine similarity: 0.7404
--------------------------------------------------------------------------------
List1: 'Machine learning is fascinating.'
Best match in List2: 'I find AI to be very interesting.'
Cosine similarity: 0.5942
--------------------------------------------------------------------------------
List1: 'Artificial intelligence is transforming the world.'
Best match in List2: 'The study of machine intelligence is evolving rapidly.'
Cosine similarity: 0.6928
--------------------------------------------------------------------------------
List1: 'Lung cancer'
Best match in List2: 'Self reported illness code or trait: lung cancer'
Cosine similarity: 0.6208
--------------------------------------------------------------------------------


In [72]:
cat_info['PUBMED ID'].nunique()

7125

In [8]:
cat_info['PUBMED ID'].values[0:10]

array([25672763, 25649651, 25672891, 25628645, 25241909, 25241909,
       25241909, 25823570, 25823570, 25826619])

In [9]:
info['PMID'].values[0:10]

array(['20732625', '30478444', '30478444', '28540026', '21926972',
       '26754954', '26754954', '22472876', '21926974', '23974872'],
      dtype=object)

In [10]:
all_pmids = [x for x in info.PMID.unique() if x.isnumeric()]

In [11]:
missing_pmids = [x for x in all_pmids if int(x) not in cat_info['PUBMED ID'].unique()]

In [12]:
all_traits = info.Trait.unique()

In [13]:
missing_triats = [x for x in all_traits if x not in cat_info['DISEASE/TRAIT'].values]

In [14]:
len(missing_triats)

4177

In [15]:
cat_info.head()

,DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,GENOTYPING TECHNOLOGY,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI,COHORT,FULL SUMMARY STATISTICS,SUMMARY STATS LOCATION,GXE
0,2015-12-16,25672763,Ueta M,2015-02-08,J Allergy Clin Immunol,www.ncbi.nlm.nih.gov/pubmed/25672763,"IKZF1, a new susceptibility gene for cold medi...",Cold medicine-related Stevens-Johnson syndrome...,"117 Japanese ancestry cases, 691 Japanese ance...","43 East Asian ancestry cases, 278 East Asian a...",...,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no
1,2016-01-12,25649651,Wang X,2015-02-03,J Alzheimers Dis,www.ncbi.nlm.nih.gov/pubmed/25649651,Genetic Determinants of Survival in Patientswi...,Alzheimer's disease (survival time),983 cases,NaN,...,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no
2,2015-11-07,25672891,Shimizu S,2015-02-11,J Dent Res,www.ncbi.nlm.nih.gov/pubmed/25672891,A genome-wide association study of periodontit...,Periodontitis,"1,593 Japanese ancestry cases, 7,980 Japanese ...","1,167 Japanese ancestry cases, 7,178 Japanese ...",...,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no
3,2016-01-05,25628645,Tong Y,2015-01-13,Front Genet,www.ncbi.nlm.nih.gov/pubmed/25628645,Identification of genetic variants or genes th...,Response to Homoharringtonine (cytotoxicity),93 African American ancestry lymphoblastoid ce...,NaN,...,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no
4,2015-05-30,25241909,Lee JH,2014-09-21,Respir Res,www.ncbi.nlm.nih.gov/pubmed/25241909,Genetic susceptibility for chronic bronchitis ...,Chronic obstructive pulmonary disease,"3,777 European ancestry cases, 3,520 European ...",NaN,...,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no


## BERT based matching

In [17]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode sentences
embedding1 = model.encode('First sentence from database A')
embedding2 = model.encode('Second sentence from database B')

# Compute cosine similarity
similarity = util.cos_sim(embedding1, embedding2)

In [18]:
similarity

tensor([[0.8693]])